#Custom Named Entity Recognition of Resume Parsing using Spacy3

In [1]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00


In [2]:
!pip install -U spacy

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.6.1'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cv_data=json.load(open('/content/drive/MyDrive/NER_RESUME_PARSING/train_data.json'))

In [7]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [8]:
#https://spacy.io/usage/training#quickstart------------> (to create base_config file)
!python -m spacy init fill-config /content/drive/MyDrive/NER_RESUME_PARSING/base_config.cfg config.cfg

2023-09-20 06:29:52.258123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# After this go to config.cfg change max_steps=1500 and eval_frequency=250

In [9]:
def get_spacy_doc(file,data):
  nlp=spacy.blank("en")
  db=DocBin()
  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']
    ents=[]
    entity_indicies=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indicies:
          skip_entity=True
          break
      if skip_entity ==True:
        continue
      entity_indicies=entity_indicies+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue
      if span is None:
          err_data=str([start,end])+"  "+str(text)+"\n"
          file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

In [10]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.3)

In [11]:
file=open('error.txt','w')

db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')
file.close()

100%|██████████| 60/60 [00:00<00:00, 82.26it/s]


In [12]:
! python -m spacy train /content/config.cfg --output ./ --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-09-20 06:30:50.778961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoi

In [13]:
nlp_best=spacy.load('/content/model-best')

In [14]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 54.4 MB/s eta 0:00:00


In [15]:
import sys, fitz

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

In [26]:
fname='/content/drive/MyDrive/surge_classes/Resumes/Venkateswari_Resume (1).pdf'

In [27]:
doc=fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())
tx = " ".join(text.split('\n'))
print(tx)

1/3      VENKATESWARI KANNETI   Vijayawada, India   + 91 9381386067   venkateswarikanneti3335@gmail.com   https://www.linkedin.com/in/venkateswari-kanneti-68a75b236      Professional Objective  Seeking for an opportunity to place in high paid job on emerging technologies that levelup my skills  and boosts my confidence.      Skill set  C, Python-Good grip on packages such as NLP,Spacy,Tensorflow  etc..good grip on mathematics involved in some models such as linear  regression,multiple linear regression,logistic regression DSA(Stacks,  Queues, Linked Lists, Trees), HTML, CSS, Java Script, PhP.    Education  Bachelor of Technology - BTech • Sagi Ramakrishnam Raju Engineer-  ing College  Computer Science • Grade: 8.8                  Aug 2020 - Present  Participation in university was essential to student life. Numerous activities were undertaken to foster  physical and spiritual development. Here, I acquired the skills to rebound after setbacks.  Intermediate • Sri Chaitanya Junior Colle

In [28]:
doc = nlp_best(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - VENKATESWARI KANNETI
LOCATION                      - Vijayawada
SKILLS                        - C, Python-Good grip on packages such as NLP,Spacy,Tensorflow  etc..good grip on mathematics involved in some models such as linear  regression,multiple linear regression,logistic regression DSA(Stacks,  Queues, Linked Lists, Trees), HTML, CSS, Java Script, PhP.    Education  Bachelor of Technology - BTech •
COLLEGE NAME                  - Sagi Ramakrishnam Raju Engineer-


In [19]:
f='/content/drive/MyDrive/surge_classes/Resumes/Alice Clark CV.pdf'
doc=fitz.open(f)
text = ""
for page in doc:
    text = text + str(page.get_text())
tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [21]:
doc = nlp_best(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

LOCATION                      - Delhi
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - Indian Institute of Technology
SKILLS                        - Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal  skills with ability to interact with individuals at all the levels  • Quick learner and maintains cordial relationship with project manager and team members and  good performer both in team and independent job environments  • Positive attitude towards superiors &amp; pe